In [6]:
pip install --upgrade transformers accelerate sentencepiece

   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 0.0/991.5 kB ? eta -:--:--
   ---------------------------------------- 991.5/991.5 kB 4.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
!pip install --quiet transformers accelerate sentencepiece

In [5]:

from transformers import pipeline


In [6]:
llama_pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", max_new_tokens=200, temperature=0.7)


c:\Users\Asus\anaconda3\envs\flas1\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--TinyLlama--TinyLlama-1.1B-Chat-v1.0. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Device set to use cpu


In [7]:
direct_prompt = "Explain the concept of photosynthesis."

few_shot_prompt = """
Q: What is the capital of France?
A: The capital of France is Paris.

Q: What is the process of the water cycle?
A: The water cycle is the continuous movement of water on, above, and below the Earth's surface through evaporation, condensation, and precipitation.

Q: Explain the concept of photosynthesis.
A:
"""

cot_prompt = """
Explain the concept of photosynthesis step-by-step.

Step 1: Plants absorb sunlight through chlorophyll in their leaves.
Step 2: Carbon dioxide from the air and water from the soil enter the plant.
Step 3: With sunlight energy, these are converted into glucose (a type of sugar) and oxygen.
Step 4: The plant uses glucose as energy and releases oxygen into the air.

Now, summarize photosynthesis:
"""


In [8]:
def generate(prompt):
    output = llama_pipe(prompt)[0]['generated_text']
    return output[len(prompt):].strip()  # Remove original prompt from output

# Run all
print("=== Direct Prompt Output ===")
print(generate(direct_prompt))

print("\n=== Few-Shot Prompt Output ===")
print(generate(few_shot_prompt))

print("\n=== Chain-of-Thought Prompt Output ===")
print(generate(cot_prompt))


=== Direct Prompt Output ===
4. What is the difference between photosynthesis and respiration? 5. How does the movement of nutrients in the plant's cytoplasm contribute to photosynthesis? 6. How does the light-dependent reactions in photosynthesis occur? 7. What is the process of starch and glycogen synthesis in photosynthesis? 8. How does the process of osmosis occur in photosynthesis? 9. What is the role of chlorophyll in photosynthesis? 10. Explain how the light-dependent reactions of photosynthesis are responsible for the production of organic compounds. 11. Describe the process of the Calvin cycle that occurs after photosynthesis. 12. What is the role of the thylakoid membrane in photosynthesis? 13. How long does it take for the light-dependent reactions to

=== Few-Shot Prompt Output ===
Photosynthesis is the process by which plants convert light energy into oxygen, water, and glucose (carbon dioxide) using ATP and NADPH as energy sources.

Q: What is the difference between a vol

In [9]:
def generate(prompt):
    output = llama_pipe(prompt)[0]['generated_text']
    # Remove the prompt text from output to get only generated continuation
    return output[len(prompt):].strip()

# Step 6: Generate outputs
direct_output = generate(direct_prompt)
few_shot_output = generate(few_shot_prompt)
cot_output = generate(cot_prompt)

In [14]:
!pip install sentence-transformers


In [15]:
from sentence_transformers import SentenceTransformer, util
embedder = SentenceTransformer('all-MiniLM-L6-v2')

c:\Users\Asus\anaconda3\envs\flas1\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Asus\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to

In [16]:
reference_answer = """
Photosynthesis is the process by which green plants use sunlight to convert carbon dioxide and water into glucose and oxygen.
"""

# Step 8: Compute embeddings for reference and outputs
ref_emb = embedder.encode(reference_answer, convert_to_tensor=True)

outputs = {
    "Direct Prompt": direct_output,
    "Few-Shot Prompt": few_shot_output,
    "Chain-of-Thought Prompt": cot_output
}

# Step 9: Print generated outputs and similarity scores
print("=== Generated Outputs and Similarity Scores ===\n")

for name, output in outputs.items():
    out_emb = embedder.encode(output, convert_to_tensor=True)
    score = util.cos_sim(ref_emb, out_emb).item()
    print(f"--- {name} ---")
    print(f"Output:\n{output}\n")
    print(f"Cosine Similarity to Reference: {score:.4f}\n")
    print("="*60 + "\n")

=== Generated Outputs and Similarity Scores ===

--- Direct Prompt ---
Output:
Who is responsible for the process? What do plants need to survive and reproduce? How does the process of photosynthesis impact the global climate?

Cosine Similarity to Reference: 0.6601


--- Few-Shot Prompt ---
Output:
Photosynthesis is the process of producing food from light energy using chlorophyll and carbon dioxide.

Q: What is the difference between terrestrial and marine biomes?
A: Terrestrial biomes are characterized by plants, while marine biomes are characterized by aquatic plants and animals.

Q: What is a pest and how are they managed?
A: Pests are organisms that damage or destroy crops and other agricultural products. They can be managed through various methods such as pest control, crop rotation, and weed control.

Cosine Similarity to Reference: 0.6057


--- Chain-of-Thought Prompt ---
Output:
Photosynthesis is a process by which plants and other organisms convert sunlight into energy, usin